In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F

from ordered_set import OrderedSet

## Data preprocessing

In [2]:
string = 'tomato'
string = list(string)
string

['t', 'o', 'm', 'a', 't', 'o']

In [3]:
set_string = list(OrderedSet(string)) # 중복되는 철자 제거
set_string

['t', 'o', 'm', 'a']

In [4]:
string_to_index = {s:idx for idx, s in enumerate(set_string)}
string_to_index

{'t': 0, 'o': 1, 'm': 2, 'a': 3}

In [5]:
# tomato onehot encoding
def one_hot_encoding(word, word_to_index):
    one_hot_vector = [0]*(len(word_to_index))
    index = word_to_index[word]
    one_hot_vector[index] = 1
    return one_hot_vector

encoding_X = []
for s in string[:-1]: # input은 마지막 철자를 제외
    encoding_X.append(one_hot_encoding(s, string_to_index))

target = [string_to_index[s] for s in string[1:]]
    
X = torch.FloatTensor(encoding_X).reshape(5,1, 4) # 실수형 텐서
Y = torch.LongTensor(target) # 정수형 텐서 

In [6]:
X.shape # (batch, Seq, input_dim)

torch.Size([5, 1, 4])

In [7]:
Y

tensor([1, 2, 3, 0, 1])

## Hyper parameters

In [8]:
input_size = 4 # 원핫인코딩의 Dimension
hidden_size = 4

num_epoch = 20
learning_rate = 0.1

## Model

In [9]:
model = torch.nn.RNN(input_size, hidden_size, batch_first=True) # batch_first => (batch, seq, input_dim)

## Loss function, Optimizer

In [10]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [11]:
for epoch in range(num_epoch):
    optimizer.zero_grad()
    
    outputs, status = model(X)
    outputs = outputs.reshape(-1, input_size)
    
    loss = loss_function(outputs, Y)
    
    loss.backward()
    optimizer.step()
    
    result = outputs.detach().numpy().argmax(axis=1)
    result_string = ''.join([set_string[s] for s in result])
    
    print(f"Epoch: {epoch+1}, loss:{loss:.2f} --> 예측한 문자: {result_string}, 예측값: {result}")

Epoch: 1, loss:1.12 --> 예측한 문자: oaato, 예측값: [1 3 3 0 1]
Epoch: 2, loss:0.99 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 3, loss:0.90 --> 예측한 문자: omoto, 예측값: [1 2 1 0 1]
Epoch: 4, loss:0.81 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 5, loss:0.74 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 6, loss:0.67 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 7, loss:0.61 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 8, loss:0.56 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 9, loss:0.53 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 10, loss:0.50 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 11, loss:0.48 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 12, loss:0.46 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 13, loss:0.44 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 14, loss:0.43 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 15, loss:0.42 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 16, loss:0.41 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 17, loss:0.40 --> 예측한 문자: omato, 예측값: [1 2 3 0 1]
Epoch: 18, loss:0.39 --> 예측한 문자: omato, 

## Test

In [12]:
test_string = list('toma')

test_X = []
for s in test_string:
    test_X.append(one_hot_encoding(s, string_to_index))
test_X = torch.FloatTensor(test_X).reshape(4, 1, 4)

In [13]:
test_X

tensor([[[1., 0., 0., 0.]],

        [[0., 1., 0., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 0., 1.]]])

In [14]:
with torch.no_grad():
    model.eval()
    test_output, status = model(test_X)
    test_output = test_output.reshape(-1, input_size)
    
    result = test_output.detach().numpy().argmax(axis=1)
    result_string = ''.join([set_string[s] for s in result])
    
    print(f"예측한 문자: {result_string}, 예측값: {result}")

예측한 문자: omat, 예측값: [1 2 3 0]
